# LLM RAG 파이프라인 테스트

이 노트북에서는 구현된 RAG (Retrieval-Augmented Generation) 파이프라인의 각 컴포넌트를 테스트하고 전체 시스템을 평가합니다.

## 목차
1. 컴포넌트 임포트
2. 환경 설정
3. LLM 초기화
4. 임베딩 생성 테스트
5. 벡터 스토어 작업 테스트
6. RAG 파이프라인 테스트
7. 대화형 쿼리 테스트

## 1. 컴포넌트 임포트
필요한 모든 모듈을 임포트합니다.

In [ ]:
# 시스템 경로 설정
import sys
import os

# 프로젝트 루트 디렉토리를 파이썬 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

# RAG 컴포넌트 임포트
from llm_rag.config import get_config
from llm_rag.rag_pipeline import get_rag_chain
from llm_rag.embeddings.embedder import get_embeddings_model
from llm_rag.embeddings.vectorstore_chroma import get_vector_store
from llm_rag.llm.llm_loader import get_llm
from llm_rag.llm.prompt_templates import QA_CHAIN_PROMPT

## 2. 환경 설정
설정 파일에서 필요한 매개변수를 로드하고 환경을 초기화합니다.

In [ ]:
# 설정 로드
config = get_config()
print("설정 로드 완료:", config)

## 3. LLM 초기화
LLM을 초기화하고 기본 응답 생성을 테스트합니다.

In [ ]:
# LLM 초기화
llm = get_llm()

# 간단한 테스트 쿼리
test_query = "Hello! Can you help me understand what you can do?"
print("테스트 쿼리:", test_query)
response = llm.predict(test_query)
print("\nLLM 응답:", response)

## 4. 임베딩 생성 테스트
텍스트 샘플에 대한 임베딩을 생성하고 시각화합니다.

In [ ]:
# 임베딩 모델 초기화
embedding_model = get_embeddings_model()

# 테스트 텍스트
test_texts = [
    "이 제품은 동물용 의약품입니다.",
    "반려동물의 건강을 위한 영양제입니다.",
    "처방이 필요한 전문 의약품입니다."
]

# 임베딩 생성
embeddings = embedding_model.embed_documents(test_texts)

print(f"생성된 임베딩 수: {len(embeddings)}")
print(f"임베딩 차원: {len(embeddings[0])}")

# 임베딩 시각화를 위한 차원 축소 (선택사항)
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

# TSNE를 사용한 2D 시각화
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(10, 6))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
for i, text in enumerate(test_texts):
    plt.annotate(text, (embeddings_2d[i, 0], embeddings_2d[i, 1]))
plt.title("텍스트 임베딩 시각화")
plt.show()

## 5. 벡터 스토어 작업 테스트
ChromaDB를 사용하여 임베딩을 저장하고 검색합니다.

In [ ]:
# 벡터 스토어 초기화
vector_store = get_vector_store()

# 테스트 데이터 추가
test_texts = [
    "강아지용 영양제로, 관절 건강에 도움을 줍니다.",
    "고양이용 항생제로, 수의사 처방이 필요합니다.",
    "반려동물용 구충제로, 6개월마다 정기적인 투여가 권장됩니다."
]

# 벡터 스토어에 데이터 추가
vector_store.add_texts(test_texts)

# 유사도 검색 테스트
query = "강아지 영양제 추천"
search_results = vector_store.similarity_search(query, k=2)

print("검색 쿼리:", query)
print("\n검색 결과:")
for i, doc in enumerate(search_results, 1):
    print(f"\n{i}. 문서 내용: {doc.page_content}")
    print(f"   유사도 점수: {doc.metadata.get('score', 'N/A')}")

## 6. RAG 파이프라인 테스트
전체 RAG 파이프라인을 테스트합니다.

In [ ]:
# RAG 체인 초기화
rag_chain = get_rag_chain()

# 테스트 쿼리
test_queries = [
    "강아지 관절 건강에 좋은 제품이 있나요?",
    "고양이 항생제는 어떻게 사용해야 하나요?",
    "반려동물 구충제 사용 주기는 어떻게 되나요?"
]

print("RAG 파이프라인 테스트 결과:")
print("=" * 50)

for query in test_queries:
    print(f"\n질문: {query}")
    response = rag_chain.invoke(query)
    print(f"응답: {response}")

## 7. 대화형 쿼리 테스트
사용자 입력을 받아 실시간으로 RAG 파이프라인을 테스트합니다.

In [ ]:
def interactive_qa():
    print("대화형 Q&A 시스템을 시작합니다.")
    print("종료하려면 'quit' 또는 'exit'를 입력하세요.\n")
    
    while True:
        query = input("\n질문을 입력하세요: ")
        if query.lower() in ['quit', 'exit']:
            print("\n대화를 종료합니다.")
            break
            
        response = rag_chain.invoke(query)
        print(f"\n응답: {response}")

# 대화형 테스트 실행
interactive_qa()